# Setup of the AnnData object
**Author:** [Severin Dicks](https://github.com/Intron7) (IBSM Freiburg)

This notebook is just downloader and sets up the AnnData object (https://anndata.readthedocs.io/en/latest/index.html) we will be working with. In this example workflow we'll be looking at a dataset of ca. 90000 cells from lungcancer patients published by [Quin et al., Cell Research 2020](https://www.nature.com/articles/s41422-020-0355-0). 

In [1]:
import gdown
import os
url = 'https://drive.google.com/uc?id=1eoK0m2ML1uNLc80L6yBuPrkJqsDF-QWj'
os.makedirs("./h5",exist_ok=True)
output = './h5/adata.raw.h5ad'
gdown.download(url, output, quiet=True)

'./h5/adata.raw.h5ad'